In [13]:
import pandas as pd
import numpy as np

data = pd.read_csv('../database/personality/data.csv', delimiter=', ')
ratings = pd.read_csv('../database/personality/ratings.csv', delimiter=', ')
movies = pd.read_csv('../database/personality/movies.csv')

/var/folders/1h/930ycynj1hldsd54hf2p__f80000gn/T/ipykernel_61641/1980232867.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('../database/personality/data.csv', delimiter=', ')
/var/folders/1h/930ycynj1hldsd54hf2p__f80000gn/T/ipykernel_61641/1980232867.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('../database/personality/ratings.csv', delimiter=', ')


In [38]:
users = data[['userid', 'openness', 'agreeableness', 'emotional_stability', 'conscientiousness', 'extraversion', 'assigned_metric', 'assigned_condition']]
valid_movie_ids = movies['movieId'].unique()
valid_ratings = ratings.loc[ratings['movieId'].isin(valid_movie_ids)]
movies_df = movies.loc[movies['movieId'].isin(valid_ratings['movieId'].unique())]

ratings_df = valid_ratings.loc[valid_ratings['userId'].isin(users['userid'].unique())]
users_df = users.loc[users['userid'].isin(ratings_df['userId'].unique())]
users_df = users_df.rename(columns={'userid': 'user_id'})

genres = movies_df['genres'].str.split('|', expand=True).stack().reset_index(level=-1, drop=True)
genres_df = pd.DataFrame(genres.unique(), columns=['genre'])

genres_df['genre_id'] = range(1, len(genres_df) + 1)
genre_to_id = genres_df.set_index('genre')['genre_id'].to_dict()
genres = genres.map(genre_to_id)

genre_movie_df = pd.DataFrame({
    'movie_id': genres.index.get_level_values(0),
    'genre_id': genres.values
})
genre_movie_df.reset_index(drop=True, inplace=True)

movies_df = movies_df.drop('genres', axis=1).rename(columns={'movieId': 'movie_id'})
ratings_df = ratings_df.drop('timestamp', axis=1).rename(columns={'userId': 'user_id', 'movieId': 'movie_id'})

return users_df, movies_df, genres_df, genre_movie_df, ratings_df

,movie_id,genre_id
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
64048,59162,20
64049,59172,20
64050,59173,8
64051,59173,9
